<h1>MCAS results: Why are some schools considered better than others?</h1>
<p>Data are from:</p>
    <p>
    MCAS achievement, MCAS growth percentiles<br>
<a href=https://profiles.doe.mass.edu/statereport/nextgenmcas.aspx>
    https://profiles.doe.mass.edu/statereport/nextgenmcas.aspx</a>

<p>Race and gender of students<br><a href=https://profiles.doe.mass.edu/statereport/enrollmentbyracegender.aspx>
    https://profiles.doe.mass.edu/statereport/enrollmentbyracegender.aspx</a>

<p>High needs/English learners/low income etc.<br><a href=https://profiles.doe.mass.edu/statereport/selectedpopulations.aspx>
    https://profiles.doe.mass.edu/statereport/selectedpopulations.aspx</a>

<p>Total per pupil expenditure<br><a href=https://profiles.doe.mass.edu/statereport/ppx.aspx>
    https://profiles.doe.mass.edu/statereport/ppx.aspx</a>

<p>Town/city income per capita<br><a href=https://dlsgateway.dor.state.ma.us/reports/rdPage.aspx?rdReport=DOR_Income_EQV_Per_Capita>
    https://dlsgateway.dor.state.ma.us/reports/rdPage.aspx?rdReport=DOR_Income_EQV_Per_Capita</a>

<p>School data are taken from the 2018-2019 school year, the last full school year before pandemic interruptions. MCAS tests were taken in the spring of 2019, and fiscal data are from fiscal year 2019.

In [1]:
#Load the data into pandas dataframes
import pandas
import pandasql
mcas=pandas.read_csv("NextGenMCAS2019.csv",encoding="utf-8",delimiter="\t")
demographics=pandas.read_csv("enrollmentbydemographic2018-2019.csv",encoding="utf-8",delimiter="\t")
student_needs=pandas.read_csv("selectedpopulations2018-2019.csv",encoding="utf-8",delimiter="\t")
ppexp=pandas.read_csv("PerPupilExpenditures_2019.csv",encoding="utf-8",delimiter="\t")
ppexp["In-District Expenditures per Pupil"]= \
ppexp["In-District Expenditures per Pupil"].apply(lambda x: float(x.replace("$","").replace(",","")))
town_income=pandas.read_csv("DOR_Income_Per_Capita_2019.csv",encoding="utf-8",delimiter="\t",thousands=",")

#Drop unused columns
ppexp=ppexp[["District Code","In-District Expenditures per Pupil"]]
town_income=town_income[["Municipality","DOR Income Per Capita","EQV Per Capita"]]
#Combine the science, math, and English Language Arts data into a combined score for each school district.
mcas_combined_subjects=pandasql.sqldf("SELECT `District Code`,MAX(`District Name`) AS district_name," +
                                      "AVG(`M+E %`) AS 'Passing %',AVG(SGP) AS 'Student Growth Percentile' " +
                                      "FROM mcas GROUP BY `District Code` ORDER BY `District Code`",globals())
#Merge the MCAS results data with data on school district demographics and spending
schools_combined=pandas.merge(mcas_combined_subjects,demographics,how="inner",on="District Code")
schools_combined=pandas.merge(schools_combined,student_needs,how="inner",on="District Code")
schools_combined=pandas.merge(schools_combined,ppexp,how="inner",on="District Code")
fulldf=pandas.merge(schools_combined,town_income,how="inner",left_on="district_name",right_on="Municipality")
df=fulldf.copy()
for column in df.columns:
    if "#" in column:
        df.pop(column)
df=df.drop(["District Name_x","District Name_y"],axis=1)

In [10]:
#Analyze the data
import plotly
import plotly.express as px
p1=px.scatter(df,x="DOR Income Per Capita",y="Passing %",log_x=True,hover_data=
              {"district_name","Passing %"},trendline="ols",trendline_options=dict(log_x=True))
p1.update_xaxes(range=[4,5.60206],dtick="D1",title="city/town income per capita (dollars/year)")
p1.show()
p2=px.scatter(df,x="In-District Expenditures per Pupil",y="Passing %",hover_data=
              {"district_name","Passing %"},trendline="ols")
p2.show()

As you can see, there is a modest correlation (R<sup>2</sup>=0.61) between per capita income and MCAS scores, but no correlation between district educational spending and MCAS performance. This suggests something other than school spending causes the higher score districts to outperform others.

In [13]:
#Linear model
import sklearn
from sklearn.linear_model import LinearRegression as LinReg
x3=df[["In-District Expenditures per Pupil"]]
y3=df["Passing %"]
Reg3=LinReg().fit(x3,y3)
print(Reg3.score(x3,y3))

0.0008530044528787251
